In [27]:
%matplotlib inline
import lightgbm as lgb
import numpy as np
import pandas as pd
import random
from scipy.stats import norm
import pickle
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import preprocessing 
from bs4 import BeautifulSoup
import re
from bayes_opt import BayesianOptimization
import graphviz
import warnings
warnings.simplefilter('ignore')
def make_Xyqodds():
    result = [[[],[],[],[]],[[],[],[],[]]]
    with open("keiba_2019",mode="r") as f:
        for line in f.readlines():
          
            year = int(line[0:4])
            if year < 2020:
                num = 0
            else:
                num = 1
        
            
            line_list = line.split(",")
            race_info,horses_info,odds_info = line_list[0],line_list[1:-1],line_list[-1]
            race_list = race_info.split(";")
            
            for horse_info in horses_info:
                horse_list = horse_info.split(";")
                label = int(18/int(horse_list[0]))
                data = race_list + horse_list[1:] +[race_list[0][4:6],race_list[0][-2:]]
                if data[10] == "--":
                    continue
                for i in [2,5,8,10,11,12,18]:
                    try:
                        data[i] = float(data[i])
                    except:
                        data[i] = 0
                result[num][0].append(data)
                result[num][1].append(label)
            
            result[num][2].append(len(horses_info))
            result[num][3].append(odds_info)
    result = np.array(result)

    return result
def train_sim(max_depth,num_leaves,max_position):
    
    boosting_type = "gbdt"
    
    result = make_Xyqodds()
    X_train,X_test = result[0][0],result[1][0]
    y_train,y_test = result[0][1],result[1][1]
    q_train,q_test = result[0][2],result[1][2]
    odds_train,odds_test = result[0][3],result[1][3]
    category = [0,1,3,4,6,7,9,13,14,15,16,17,19,20]
    number =[2,5,8,10,11,12,18]
    X_train = pd.DataFrame(X_train)
    X_test = pd.DataFrame(X_test)
    for i in category:
        X_train[i] = X_train[i].astype('category')
        X_test[i] = X_test[i].astype('category')
    #for i in number:
        #X_train[i] = X_train[i].astype('float')
        #X_test[i] = X_test[i].astype('float')
    #columns_name = []
    
    X_train.drop(columns=10,inplace=True) 
    X_test.drop(columns=10,inplace=True) 

    train = lgb.Dataset(X_train, y_train,group=q_train)
    test = lgb.Dataset(X_test, y_test, group=q_test)

    params = { 
        'objective': 'lambdarank',
        'metric': 'ndcg',
        'ndcg_eval_at': 3,
        'max_depth': int(max_depth),
        'max_position':int(max_position), 
        'num_leaves': int(num_leaves),
        'min_data_in_leaf':1,
        'verbose': -1,
        'categorical_feature':category,
        'boosting_type': boosting_type
    }
    #print(type(X_train.loc[0,2]))
    model = lgb.train(
        params,
        train,
        num_boost_round=150,
        valid_sets=[train,test],
        valid_names=['train','test'],
        early_stopping_rounds=20,
        verbose_eval=True
    )

    from_ = 0
    odds_all = odds_test
   
    
    gain = [[],[],[],[],[],[],[]]
    hit = [0,0,0,0,0,0,0]
    
    for i,odds in zip(q_test,odds_all):
        pred = model.predict(X_test[from_:from_+i],num_iteration=model.best_iteration)
        pred = pd.Series(pred).sort_values(ascending=False)          
        pred.index = pred.index + 1
        pred_rank = list(map(str,pred.index.values))[0:3]

        from_ += i
        odds_list = re.findall(r'([^\|]*)\|([^\|]*)\|([^;]*)円;?',odds)

        result = []
        for i,j in zip([1,2],[" ","円"]):
            comb_list =  [x[i] for x in odds_list if x[0] != "枠連"]
            comb_list[0] = [comb_list[0:1]]
            comb_list[1] = [[a] for a in comb_list[1].split(j)]
            if len(comb_list[1]) == 5:
                comb_list[1] = comb_list[1][::2]
                j = "   "
            comb_list[2] = [set(comb_list[2].split(j))]
            comb_list[3] = comb_list[3].split(j)
            comb_list[3] = [set(comb_list[3][0:(3-i)]),set(comb_list[3][3-i:6-2*i]),set(comb_list[3][6-2*i:9-3*i])]
            comb_list[4] = [comb_list[4].split(j)]
            comb_list[5] = [set(comb_list[5].split(j))]
            comb_list[6] = [comb_list[6].split(j)]
            result.append(comb_list)

        baken_list = [0,0,0,0,0,0,0,0]
        baken_list[0] = [[a] for a in pred_rank[0:1]]
        baken_list[1] = [[a] for a in pred_rank[0:3]]
        baken_list[2] = [set(pred_rank[0:2])]
        baken_list[3] = [set(pred_rank[0:2]),set(pred_rank[1:3]),set(pred_rank[::2])]
        baken_list[4] = [pred_rank[0:2]]
        baken_list[5] = [set(pred_rank)]
        baken_list[6] = [pred_rank]
        
        for k in range(len(result[0])):
            for j in range(len(result[0][k])):
                if result[0][k][j] in baken_list[k]:
                    gain[k].append(float(list(result[1][k][j])[0]))
                    hit[k] += 1
                    
                else:
                    gain[k].append(0)
                


 
    gain = np.array(gain)
    objective = []
    for i in range(7):
        p = norm.sf(x=100,loc=np.average(gain[i]),scale=np.std(gain[i],ddof=1)/np.sqrt(len(gain[i])))
        print(p,hit[i]/len(gain[i]),np.average(gain[i]),np.std(gain[i],ddof=1)/np.sqrt(len(gain[i])))
        
    importance = pd.DataFrame(model.feature_importance(), index=X_train.columns.values, columns=['importance'])
    display(importance)
    return model,gain

In [28]:
import pickle
model,gain = train_sim(9.853,904.1,1.045) #max_depth,num_leaves,max_position




[1]	train's ndcg@3: 0.380809	test's ndcg@3: 0.305741
Training until validation scores don't improve for 20 rounds
[2]	train's ndcg@3: 0.458242	test's ndcg@3: 0.286655
[3]	train's ndcg@3: 0.492718	test's ndcg@3: 0.295532
[4]	train's ndcg@3: 0.516398	test's ndcg@3: 0.309584
[5]	train's ndcg@3: 0.534092	test's ndcg@3: 0.316888
[6]	train's ndcg@3: 0.542101	test's ndcg@3: 0.324925
[7]	train's ndcg@3: 0.550646	test's ndcg@3: 0.326523
[8]	train's ndcg@3: 0.564318	test's ndcg@3: 0.321566
[9]	train's ndcg@3: 0.578622	test's ndcg@3: 0.325075
[10]	train's ndcg@3: 0.586874	test's ndcg@3: 0.325523
[11]	train's ndcg@3: 0.59087	test's ndcg@3: 0.332098
[12]	train's ndcg@3: 0.597695	test's ndcg@3: 0.338755
[13]	train's ndcg@3: 0.606948	test's ndcg@3: 0.337614
[14]	train's ndcg@3: 0.614773	test's ndcg@3: 0.339735
[15]	train's ndcg@3: 0.621405	test's ndcg@3: 0.346274
[16]	train's ndcg@3: 0.62584	test's ndcg@3: 0.3477
[17]	train's ndcg@3: 0.629886	test's ndcg@3: 0.344697
[18]	train's ndcg@3: 0.634153	test

,importance
0,2
1,746
2,1720
3,723
4,729
5,1332
6,68
7,161
8,756
9,116


In [29]:
pickle.dump(model, open('trained_model.pkl', 'wb'))

In [25]:
a = []
limit = 100
l = np.random.normal(loc=155.78947368421052,scale=46.65859374565609,size=limit)/100
m = 1
for i in l:
    if i < 0:
        i = 0
    m *= i
a.append(m)
print(np.average(a),np.std(a,ddof=1)/np.sqrt(limit))
    

3.291307879799485e+17 nan


In [26]:
money = 10000
for get in gain[4]:
    bet = 100
    money -= bet
    if money <= 0:
        break
    money += bet*get/100
print(money,len(gain[4]))
    

39680.0 532


In [86]:
3//2

1